In [31]:
import pandas as pd
import numpy as np
%matplotlib inline
import sklearn.model_selection as cv
import sklearn

In [32]:
data = pd.read_csv("gender-classifier-DFE-791531.csv", skipinitialspace=True)
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1,yes,1,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0.0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964.0,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1,yes,1,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0.0,C0DEED,���It felt like they were my friends and I was...,NaN,7471.0,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,"0,6625",yes,1,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1.0,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617.0,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1,yes,1,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0.0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693.0,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1,yes,1,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0.0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462.0,10/26/15 12:40,6.587300e+17,NaN,NaN


In [33]:
# Drop attributes that we won't use
data = data.drop(['_golden', '_unit_state', '_trusted_judgments', '_last_judgment_at', 'gender:confidence', 'profile_yn', 'profile_yn:confidence', 'gender_gold', 'profile_yn_gold', 'profileimage', 'tweet_id'], axis=1);
data.head()

,_unit_id,gender,created,description,fav_number,link_color,name,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_location,user_timezone
0,815719226,male,12/5/13 1:48,i sing my own rhythm.,0.0,08C2C2,sheezy0,0.0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964.0,10/26/15 12:40,main; @Kan1shk3,Chennai
1,815719227,male,10/1/12 13:51,I'm the author of novels filled with family dr...,68.0,0084B4,DavdBurnett,0.0,C0DEED,���It felt like they were my friends and I was...,NaN,7471.0,10/26/15 12:40,NaN,Eastern Time (US & Canada)
2,815719228,male,11/28/14 11:30,louis whining and squealing and all,7696.0,ABB8C2,lwtprettylaugh,1.0,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617.0,10/26/15 12:40,clcncl,Belgrade
3,815719229,male,6/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202.0,0084B4,douggarland,0.0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693.0,10/26/15 12:40,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,female,4/16/14 13:23,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318.0,3B94D9,WilfordGemma,0.0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462.0,10/26/15 12:40,NaN,NaN


In [34]:
# Description attr treatment. Change to number of words writed
for index, row in data.iterrows() :
    data.set_value(index, 'description', len(str(row['description'])))
    data.set_value(index, 'text', len(str(row['text'])))
    data.set_value(index, 'tweet_coord', "false" if pd.isnull(row['tweet_coord']) else "true")

data.rename(columns={'description': 'description_length', 'text': 'text_length'}, inplace=True)
data.head()

,_unit_id,gender,created,description_length,fav_number,link_color,name,retweet_count,sidebar_color,text_length,tweet_coord,tweet_count,tweet_created,tweet_location,user_timezone
0,815719226,male,12/5/13 1:48,21,0.0,08C2C2,sheezy0,0.0,FFFFFF,109,false,110964.0,10/26/15 12:40,main; @Kan1shk3,Chennai
1,815719227,male,10/1/12 13:51,62,68.0,0084B4,DavdBurnett,0.0,C0DEED,138,false,7471.0,10/26/15 12:40,NaN,Eastern Time (US & Canada)
2,815719228,male,11/28/14 11:30,35,7696.0,ABB8C2,lwtprettylaugh,1.0,C0DEED,80,false,5617.0,10/26/15 12:40,clcncl,Belgrade
3,815719229,male,6/11/09 22:39,146,202.0,0084B4,douggarland,0.0,C0DEED,138,false,1693.0,10/26/15 12:40,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,female,4/16/14 13:23,160,37318.0,3B94D9,WilfordGemma,0.0,0,92,false,31462.0,10/26/15 12:40,NaN,NaN


In [35]:
data[data['tweet_coord'] == "true"]

,_unit_id,gender,created,description_length,fav_number,link_color,name,retweet_count,sidebar_color,text_length,tweet_coord,tweet_count,tweet_created,tweet_location,user_timezone
81,815719307,female,7/29/08 3:50,174,6805.0,FF0877,smizz,0.0,FFFFFF,140,true,30903.0,10/26/15 12:40,Practicing Kindness,London
82,815719308,brand,8/31/12 5:36,138,0.0,F62217,TCGJobs,0.0,C0DEED,136,true,19879.0,10/26/15 12:40,NaN,Atlantic Time (Canada)
177,815719403,male,5/3/09 9:37,87,33.0,CC3366,JLJ167,0.0,DBE9ED,126,true,418.0,10/26/15 12:40,"43.122580,-87.914122",NaN
228,815719454,brand,7/12/09 8:40,22,0.0,8A1C3B,PhoenixCP,0.0,FFFFFF,138,true,141186.0,10/26/15 12:40,Phoenix,Mountain Time (US & Canada)
250,815719476,brand,8/1/09 18:53,27,0.0,57113B,IndianapolisCP,0.0,FFFFFF,142,true,211477.0,10/26/15 12:40,"Indianapolis, IN",Eastern Time (US & Canada)
470,815719697,brand,7/18/09 9:57,52,1.0,8A1C3B,PhiladelphiaCP,0.0,FFFFFF,140,true,251352.0,10/26/15 12:40,Philadelphia,Eastern Time (US & Canada)
480,815719707,brand,8/1/09 18:53,27,0.0,57113B,IndianapolisCP,0.0,FFFFFF,141,true,211477.0,10/26/15 12:40,"Indianapolis, IN",Eastern Time (US & Canada)
540,815719768,brand,7/12/09 8:40,22,0.0,8A1C3B,PhoenixCP,0.0,FFFFFF,141,true,141186.0,10/26/15 12:40,Phoenix,Mountain Time (US & Canada)
705,815719934,brand,12/3/13 13:13,154,0.0,1D7B2C,CricketCareers,0.0,FFFFFF,115,true,266.0,10/26/15 12:40,"Atlanta, Georgia",Central Time (US & Canada)
920,815720151,female,10/13/10 6:26,3,2.0,560355,xoxo11_asuka,0.0,0AB0F1,129,true,983.0,10/26/15 12:40,"35.728851,140.017265",Tokyo
